# Binarized Neural Network

The following code is the implementation used to train a Binarized neural network by encoding it as a MaxSAT problem.

The training is based on finding the best W such that an input $x \in \{-1,1\}^n$ is mapped in its corresponding output $y \in \{-1,1\}$.

In [1]:
from pysat.formula import WCNF
from pysat.examples.rc2 import RC2
import itertools
import numpy as np
np.random.seed(1)

## Generating dataset

In order to generate a dataset, after having chosen the number of features to use (`n_var`), will be created all the combinations of $\{-1,1\}^\text{n_var}$, concatenated 4 times, shuffled, and finally the corresponding target will be associated by using a boolean function (the one that the BNN will have to approximate)

In [28]:
n_var = 7
all_comb = list(itertools.product([-1, 1], repeat=n_var)) * 4
np.random.shuffle(all_comb)
training_set = all_comb[:int(len(all_comb)*0.6)]
test_set = all_comb[int(len(all_comb)*0.6):]
# 1 if (x[0] == 1 or x[1] == 1 or x[2] == 1) and (x[3] == 1 or x[4] == -1) and (x[5] == -1 or x[6] == 1) else -1 #49.22%
# 1 if (x[0] == 1 and x[1] == 1 or x[2] == 1) and (x[3] == 1 or x[4] == -1) and (x[5] == -1 or x[6] == 1) else -1 #35.16%
# 1 if (x[0] == 1 and x[1] == 1 or x[2] == 1) or (x[3] == 1 and x[4] == -1) and (x[5] == -1 or x[6] == 1) else -1 #30.47%
# 1 if (x[0] == 1 and x[1] == 1 and x[2] == 1) or (x[3] == 1 or x[4] == -1) and (x[5] == -1 and  x[6] == 1) else -1 #28.91%
# 1 if (x[0] == 1 and x[1] == 1 and x[2] == 1) and (x[3] == 1 or x[4] == -1) or (x[5] == -1 and  x[6] == 1) else -1 #32.03%
#1 if (x[0] == 1 and x[1] == 1 and x[2] == 1) and (x[3] == 1 and x[4] == -1) or (x[5] == -1 and  x[6] == 1) else -1 #27.34%
#1 if (x[0] == 1 or x[1] == 1 and x[2] == 1) and (x[3] == 1 or x[4] == -1) and (x[5] == -1 and  x[6] == 1) and (x[6] == -1 or x[0] == 1) else -1 # 09.38
#1 if (x[0] == 1 and x[1] == 1 and x[2] == 1) or (x[3] == 1 and x[4] == -1) and (x[5] == -1 or  x[6] == 1) or (x[6] == -1 and x[0] == 1) else -1 # 45.31
dec_func = lambda x : 1 if (x[0] == 1 or x[1] == 1 and x[2] == 1) and (x[3] == 1 or x[4] == -1) and (x[5] == -1 and  x[6] == 1) and (x[6] == -1 or x[0] == 1) else -1 # 09.38
train_targets = [
    dec_func(x) for x in training_set
]
test_targets = [
    dec_func(x) for x in test_set
]
if len(training_set[0]) % 2 == 0:
    training_set = np.hstack((training_set, np.ones((len(training_set), 1)))).tolist()

The following code will print the distribution of the targets in the training and test set

In [29]:
print("TRAINING set:")
print(f"{np.sum(np.array(train_targets) == 1) / len(train_targets) * 100:05.2f}% are 1\n{np.sum(np.array(train_targets) == -1) / len(train_targets) * 100:05.2f}% are -1")
print("TEST set:")
print(f"{np.sum(np.array(test_targets) == 1) / len(test_targets) * 100:05.2f}% are 1\n{np.sum(np.array(test_targets) == -1) / len(test_targets) * 100:05.2f}% are -1")

TRAINING set:
10.75% are 1
89.25% are -1
TEST set:
07.32% are 1
92.68% are -1


In [30]:
assert len(training_set) == len(train_targets) and len(training_set) > 0 and len(training_set[0]) > 0

With the following functions, all the training set and the weights will be mapped to single numbers in $\mathbb{N}^+ \slash \{0\}$

In [31]:
# get the index representing the j-th feature of the sample i
def get_index_x_i_j(i, j, padding=0):
    global training_set
    # first len(training_set[0]) variables are the weights
    return i * (len(training_set[0]) + padding) + j + len(training_set[0]) + padding + 1
# get the index representing the i-th weight
def get_index_weight_i(i):
    return i + 1

## Creating model

The following code is used to store WCNF with CNF as soft clauses, in order to be able to associate a weight to a whole CNF, and not only to a specific disjunction

In [32]:
class GroupWCNF(object):
    def __init__(self):
        self.nv = 0
        self.hard = []
        self.soft = []
        self.wght = []

    def append(self, clause, weight=None):
        if not isinstance(clause, list):
            clause = [clause]
        for subclause in clause:
            if isinstance(subclause, list):
                self.nv = max([abs(l) for l in subclause] + [self.nv])
            else:
                self.nv = max(abs(subclause), self.nv)
        if weight:
            self.soft.append(list(clause))
            self.wght.append(weight)
        else:
            self.hard.append(list(clause))

    def toWCNF(self):
        new_soft_clauses = []
        new_hard_clauses = self.hard[:]
        max_var = self.nv # max_var will be the new variable introduced
        for clause in self.soft:
            max_var+=1
            new_soft_clauses.append([max_var])
            for subclause in clause:
                new_hard_clauses.append(subclause + [-max_var])
        wcnf = WCNF()

        wcnf.extend(new_hard_clauses)
        wcnf.extend(new_soft_clauses, weights=self.wght)

        return wcnf

In [33]:
nn = GroupWCNF()

adding the hard clauses (encoding the dataset):

In [34]:
for i in range(len(training_set)):
    for j in range(len(training_set[0])):
        if training_set[i][j] > 0:
            nn.append([get_index_x_i_j(i,j)])
        else:
            nn.append([-get_index_x_i_j(i,j)])

adding the soft clauses (for each target that the model can predict correctly, will gain weight 1):

In [35]:
%%time
for i in range(len(train_targets)):
    J = itertools.combinations(range(len(training_set[0])), len(training_set[0]) // 2 + 1)
    temp = []
    if train_targets[i] < 0 :
        for x_i_subset in J:
            for comb in itertools.product([-1, 1], repeat=len(x_i_subset)):
                temp.append(
                    [get_index_x_i_j(i, x_i_subset[idx]) * _not for idx, _not in enumerate(comb)] +
                    [get_index_weight_i(x_i_subset[idx]) * _not for idx, _not in enumerate(comb)],
                )
    elif train_targets[i] > 0:
        for x_i_subset in J:
            for comb in itertools.product([-1, 1], repeat=len(x_i_subset)):
                temp.append(
                    [get_index_x_i_j(i, x_i_subset[idx]) * _not for idx, _not in enumerate(comb)] +
                    [get_index_weight_i(x_i_subset[idx]) * (_not * -1) for idx, _not in enumerate(comb)]
                )
    nn.append(temp, weight=1)

CPU times: user 425 ms, sys: 10.8 ms, total: 435 ms
Wall time: 435 ms


## Search solution

`RC2` will be used as solver since it outperforms `FM` in almost all the cases

In [36]:
solver = RC2(nn.toWCNF(), verbose=0)

In [37]:
%%time
model = solver.compute()

CPU times: user 916 ms, sys: 8.3 ms, total: 924 ms
Wall time: 962 ms


## Evaluation

In [38]:
from sklearn.metrics import accuracy_score

In [39]:
w = np.sign([model[0:len(training_set[0])]])

In [40]:
predict_train = np.sign(np.multiply(training_set, w).sum(1))
predict_test = np.sign(np.multiply(test_set, w).sum(1))
print(f"Accuracy in training: {accuracy_score(train_targets, predict_train)}")
print(f"Accuracy in test: {accuracy_score(test_targets, predict_test)}")

Accuracy in training: 0.6221498371335505
Accuracy in test: 0.551219512195122


### Brute forcing weights

As an alternative, the weights can be brute forced (MaxSAT is NP hard, therefore the worst case complexity is the same).

Thanks to the fact that Numpy uses multi threading, this approach is faster when dealing with more than 8 features, or when the target classes are very unbalanced

In [41]:
%%time
best = None
best_acc = 0
for w in itertools.product([-1,1], repeat=len(training_set[0])):
    predict_train = np.sign(np.multiply(training_set, w).sum(1))
    acc = accuracy_score(train_targets, predict_train)
    if best is None or acc > best_acc:
        best = w
        best_acc = acc

predict_test = np.sign(np.multiply(test_set, best).sum(1))
print(f"Accuracy in training: {best_acc}")
print(f"Accuracy in test: {accuracy_score(test_targets, predict_test)}")

Accuracy in training: 0.6221498371335505
Accuracy in test: 0.551219512195122
CPU times: user 30.7 ms, sys: 1.53 ms, total: 32.2 ms
Wall time: 31.3 ms


## Using Biases

The following section will try to improve the performances of the BNN when dealing with unbalanced classes, by adding biases

In [42]:
biases_count = int(len(training_set[0]) / 2)
biases_count = biases_count + (0 if (len(training_set[0]) + biases_count) % 2 else 1)
training_set_bias = np.hstack((training_set, np.ones((len(training_set), biases_count)))).tolist()
test_set_bias = np.hstack((test_set, np.ones((len(test_set), biases_count)))).tolist()

In [45]:
%%time
nn = GroupWCNF()
for i in range(len(training_set_bias)):
    for j in range(len(training_set_bias[0])):
        if training_set_bias[i][j] > 0:
            nn.append([get_index_x_i_j(i,j, padding=biases_count+1)])
        else:
            nn.append([-get_index_x_i_j(i,j, padding=biases_count+1)])

for i in range(len(train_targets)):
    J = itertools.combinations(range(len(training_set_bias[0])), len(training_set_bias[0]) // 2 + 1)
    temp = []
    if train_targets[i] < 0 :
        for x_i_subset in J:
            for comb in itertools.product([-1, 1], repeat=len(x_i_subset)):
                temp.append(
                    [get_index_x_i_j(i, x_i_subset[idx], padding=biases_count+1) * _not for idx, _not in enumerate(comb)] +
                    [get_index_weight_i(x_i_subset[idx]) * _not for idx, _not in enumerate(comb)],
                )
    elif train_targets[i] > 0:
        for x_i_subset in J:
            for comb in itertools.product([-1, 1], repeat=len(x_i_subset)):
                temp.append(
                    [get_index_x_i_j(i, x_i_subset[idx], padding=biases_count+1) * _not for idx, _not in enumerate(comb)] +
                    [get_index_weight_i(x_i_subset[idx]) * (_not * -1) for idx, _not in enumerate(comb)]
                )
    nn.append(temp, weight=1)
solver = RC2(nn.toWCNF(), verbose=0)
model = solver.compute()

CPU times: user 1min 3s, sys: 1.52 s, total: 1min 4s
Wall time: 1min 4s


In [46]:
w = np.sign([model[0:len(training_set_bias[0])]])
predict_train = np.sign(np.multiply(training_set_bias, w).sum(1))
predict_test = np.sign(np.multiply(test_set_bias, w).sum(1))
print(f"Accuracy in training: {accuracy_score(train_targets, predict_train)}")
print(f"Accuracy in test: {accuracy_score(test_targets, predict_test)}")

Accuracy in training: 0.9153094462540716
Accuracy in test: 0.9317073170731708


### Brute forcing weights

In [43]:
%%time
best = None
best_acc = 0
for w in itertools.product([-1,1], repeat=len(training_set_bias[0])):
    predict_train = np.sign(np.multiply(training_set_bias, w).sum(1))
    acc = accuracy_score(train_targets, predict_train)
    if best is None or acc > best_acc:
        best = w
        best_acc = acc

predict_test = np.sign(np.multiply(test_set_bias, best).sum(1))
print(f"Accuracy in training: {best_acc}")
print(f"Accuracy in test: {accuracy_score(test_targets, predict_test)}")

Accuracy in training: 0.9153094462540716
Accuracy in test: 0.9317073170731708
CPU times: user 498 ms, sys: 4.14 ms, total: 502 ms
Wall time: 519 ms


## Comparison with logistic regression

In order to have a better understanding of how difficult is the prediction, the following code will train a Logistic Regression model in order to have a benchmark

In [44]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(training_set, train_targets)

print(f"Accuracy in training: {clf.score(training_set, train_targets)}")
print(f"Accuracy in test: {clf.score(test_set, test_targets)}")

Accuracy in training: 1.0
Accuracy in test: 1.0
